In [80]:
from langchain import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

llm = OpenAI(
    openai_api_base=os.environ['OPENAI_REVERSE_PROXY'],
    openai_api_key=os.environ['OPENAI_API_KEY'],
    temperature=0,
    model_name='text-davinci-003'
)

In [81]:
# Next we wiil initialize a calculator tool using an LLMMathChain

from langchain.chains import LLMMathChain, LLMChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm=llm, verbose=True)



#intitialize the math tool

math_tool = Tool(
    name='calculator',
    func=llm_math.run,
    description="Userful for when you need to answer questions about math"
)

# When giving tools to LLM, we must pass as list of tools

tools = [math_tool]

In [3]:
tools[0].name, tools[0].description

('calculator', 'Userful for when you need to answer questions about math')

In [82]:
from langchain.agents import load_tools

tools = load_tools(['llm-math'],llm=llm)

In [83]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools = tools, 
    llm = llm,
    verbose= True,
    max_iterations = 3
)

In [84]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [7]:
(4.5*2.1)**2.2

139.94261298333066

In [8]:
# Lets try this with zero shot agent
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
 I need to figure out how many apples are in the boxes
Action: Calculator
Action Input: 8 * 2.5
Observation: Answer: 20.0
Thought: I need to add the apples Mary has to the apples in the boxes
Action: Calculator
Action Input: 4 + 20.0
Observation: Answer: 24.0
Thought: I now know the final answer
Final Answer: We have 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'We have 24 apples.'}

In [ ]:
# But what happens when we ask it a question from non math

zero_shot_agent("what is the capital of Norway?")


#Clearly calculator wont work for this
#Because the agent is forcing us to use the calculator tool which is not applicable at all      

In [13]:
# So as a workaround we can implement is to just add another tool that can answer this question


from langchain.prompts import PromptTemplate
from langchain.chains import  LLMChain


prompt = PromptTemplate(
    input_variables=["query"],
    template = "{query}"

)

llm_chain = LLMChain(llm=llm, prompt=prompt)


#initialize the LLM tool

llm_tool = Tool(
    name = "Language Model",
    func = llm_chain.run,
    description="use this tool for general purpose queries and logic"
)

In [14]:
#append the tool

tools.append(llm_tool)

In [15]:
#Now initialize the agent with our two tools

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)


In [18]:
zero_shot_agent("Who designed the Victoria Memorial, Kolkata?Find the square of the age of the person when he designed the Victoria Memorial, Kolkata")



> Entering new AgentExecutor chain...
 I need to find out who designed the Victoria Memorial and their age when they designed it.
Action: Language Model
Action Input: Who designed the Victoria Memorial, Kolkata?
Observation: 

The Victoria Memorial was designed by Sir William Emerson.
Thought: I need to find out how old Sir William Emerson was when he designed the Victoria Memorial.
Action: Language Model
Action Input: How old was Sir William Emerson when he designed the Victoria Memorial?
Observation: 

Sir William Emerson was 63 years old when he designed the Victoria Memorial in Calcutta, India in 1906.
Thought: I now know the age of the person when he designed the Victoria Memorial, so I can use the calculator to find the square of the age.
Action: Calculator
Action Input: 63^2
Observation: Answer: 3969
Thought:

> Finished chain.


{'input': 'Who designed the Victoria Memorial, Kolkata?Find the square of the age of the person when he designed the Victoria Memorial, Kolkata',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [19]:
!pip install -qU langchain openai google-search-results wikipedia sqlalchemy

In [20]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [21]:
from langchain import OpenAI

llm = OpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    openai_api_base=os.environ['OPENAI_REVERSE_PROXY'],
    temperature=0
)

In [22]:

from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

# With all of that set up, lets jump into Agents.

## What is an agent?

In [29]:
# We will use the agents to interact with a small sample database of stocks. We will not dive into the details because this is just a dummy tool we will build for illustrative purposes. Let's create it.


from sqlalchemy import MetaData

metadata_obj = MetaData()

In [30]:
from sqlalchemy import Column, Integer, String, Table, Date, Float


stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [35]:
from sqlalchemy import  create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [36]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [37]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id = obs[0],
    stock_ticker = obs[1],
    price = obs[2],
    date = obs[3]
    )


    with engine.begin() as conn:
        conn.execute(stmt)

In [38]:
for obs in observations:
    insert_obs(obs)

In [42]:
!pip install langchain_experimental

In [63]:
from langchain.chat_models import ChatVertexAI
llm = ChatVertexAI(temperature=0)

In [64]:
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [44]:
db = SQLDatabase(engine=engine)

In [95]:
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

/home/susearc/gamedisk/stable-torch/lib/python3.9/site-packages/langchain_experimental/sql/base.py:66: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [96]:
from langchain.agents import Tool
sql_tool  = Tool(
    name='Stock DB',
    func = sql_chain.run,
    description="Userful for when you need to answer questions about stocks and their prices"
)

Agent types

In [97]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [98]:
tools.append(sql_tool)

In [100]:
# As the name suggests we will use this agent to perform zero shot tasks on the input.In other words, this agent will have no memory.


from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)

In [101]:
#Now lets check it out

result = count_tokens(
    zero_shot_agent, 
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " + 
    "between the same stock prices in January the 4th?"
)



> Entering new AgentExecutor chain...
 I need to compare the stock prices of 'ABC' and 'XYZ' on two different days
Action: Stock DB
Action Input: Stock prices of 'ABC' and 'XYZ' on January 3rd and January 4th

> Entering new SQLDatabaseChain chain...
Stock prices of 'ABC' and 'XYZ' on January 3rd and January 4th
SQLQuery:SELECT stock_ticker, price, date FROM stocks WHERE stock_ticker IN ('ABC', 'XYZ') AND date BETWEEN '2023-01-03' AND '2023-01-04'
SQLResult: [('ABC', 232.0, '2023-01-03'), ('ABC', 225.0, '2023-01-04'), ('XYZ', 798.0, '2023-01-03'), ('XYZ', 795.0, '2023-01-04')]
Answer:On January 3rd, ABC was 232.0 and XYZ was 798.0. On January 4th, ABC was 225.0 and XYZ was 795.0.
> Finished chain.

Observation: On January 3rd, ABC was 232.0 and XYZ was 798.0. On January 4th, ABC was 225.0 and XYZ was 795.0.
Thought: I need to calculate the ratio between the two stock prices on each day
Action: Calculator
Action Input: 232.0/798.0 and 225.0/795.0
Observation: Answer: 0.082281174634699

In [102]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
Stock DB: Userful for when you need to answer questions about stocks and their prices

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, Stock DB]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


## Agent type 2: Conversational React

In [103]:
tools =  load_tools(
    ['llm-math'],
    llm=llm
)

In [104]:
tools.append(sql_tool)

In [105]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

In [106]:
conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)


In [107]:
result= count_tokens(
    conversational_agent,
    "Please provide me the stock prices for ABC on January the 1st"
)



> Entering new AgentExecutor chain...



Thought: Do I need to use a tool? Yes
Action: Stock DB
Action Input: ABC on January the 1st

> Entering new SQLDatabaseChain chain...
ABC on January the 1st
SQLQuery:SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-01'
SQLResult: [(200.0,)]
Answer:The price of ABC on January the 1st was 200.0.
> Finished chain.

Observation: The price of ABC on January the 1st was 200.0.
Thought: Do I need to use a tool? No
AI: Is there anything else I can help you with?

> Finished chain.
Spent a total of 1833 tokens


In [108]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [109]:
result = count_tokens(
    conversational_agent, 
    "What is the multiplication of the ratio of the prices of stocks 'ABC' and 'XYZ' in January 3rd and the ratio of the same prices of the same stocks in January the 4th?"
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: The multiplication of the ratio of the prices of stocks 'ABC' and 'XYZ' in January 3rd and the ratio of the same prices of the same stocks in January the 4th

ValueError: LLMMathChain._evaluate("
(ABC_Jan3 / XYZ_Jan3) * (ABC_Jan4 / XYZ_Jan4)
") raised error: 'ABC_Jan3'. Please try again with a valid numerical expression

In [110]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [111]:


docstore_agent = initialize_agent(
    tools, 
    llm, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=3
)



In [112]:
count_tokens(docstore_agent, "What were Archimedes' last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes and find his last words.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (, ARK-ihm-EE-deez; c. 287 – c. 212 BC) was an Ancient Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.

{'input': "What were Archimedes' last words?",
 'output': 'Do not disturb my circles'}

In [113]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key='api_key')
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)


In [114]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [11]:
dotenv.set_key(".env","SERPER_API_KEY",'60b7d4453835deaa6fcbf909f1759121f03b91132336a91bb660a624d137e31e')

(True,
 'SERPER_API_KEY',
 '60b7d4453835deaa6fcbf909f1759121f03b91132336a91bb660a624d137e31e')

In [32]:
from importlib import reload
reload(os)
reload(dotenv)

import dotenv
from dotenv import load_dotenv
load_dotenv()


True